## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets

#Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database/WeatherPy_database2.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.26,72,76,12.10,light rain
1,1,Kaitangata,NZ,-46.2817,169.8464,53.92,81,16,2.86,few clouds
2,2,Poum,NC,-20.2333,164.0167,81.57,78,94,13.53,overcast clouds
3,3,Carnarvon,AU,-24.8667,113.6333,77.07,88,3,5.75,clear sky
4,4,Kudahuvadhoo,MV,2.6708,72.8944,81.55,78,100,10.83,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 67.5
What is your desired maximum temperature for your trip? 89.3


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
subset_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
subset_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.26,72,76,12.10,light rain
2,2,Poum,NC,-20.2333,164.0167,81.57,78,94,13.53,overcast clouds
3,3,Carnarvon,AU,-24.8667,113.6333,77.07,88,3,5.75,clear sky
4,4,Kudahuvadhoo,MV,2.6708,72.8944,81.55,78,100,10.83,overcast clouds
9,9,Mar Del Plata,AR,-38.0023,-57.5575,69.82,61,0,9.22,clear sky
10,10,Tiarei,PF,-17.5333,-149.3333,86.02,77,81,15.86,light rain
11,11,Nong Chik,TH,6.8436,101.1780,83.08,91,98,7.90,overcast clouds
15,15,Mogadishu,SO,2.0371,45.3438,77.43,82,100,17.81,overcast clouds
17,17,Luganville,VU,-15.5333,167.1667,80.06,86,100,10.45,overcast clouds
21,21,Salinopolis,BR,-0.6136,-47.3561,78.91,85,100,7.74,light rain


In [7]:
# 4a. Determine if there are any empty rows.
subset_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = subset_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.26,72,76,12.10,light rain
2,2,Poum,NC,-20.2333,164.0167,81.57,78,94,13.53,overcast clouds
3,3,Carnarvon,AU,-24.8667,113.6333,77.07,88,3,5.75,clear sky
4,4,Kudahuvadhoo,MV,2.6708,72.8944,81.55,78,100,10.83,overcast clouds
9,9,Mar Del Plata,AR,-38.0023,-57.5575,69.82,61,0,9.22,clear sky
...,...,...,...,...,...,...,...,...,...,...
665,665,Kindu,CD,-2.9500,25.9500,70.63,98,100,2.71,overcast clouds
672,672,Malakal,SS,9.5334,31.6605,84.25,16,0,7.76,clear sky
673,673,Zemio,CF,5.0314,25.1361,79.50,25,100,3.91,overcast clouds
681,681,Chiredzi,ZW,-21.0500,31.6667,73.27,47,0,4.05,clear sky


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,78.26,-23.1203,-134.9692,light rain,
2,Poum,NC,81.57,-20.2333,164.0167,overcast clouds,
3,Carnarvon,AU,77.07,-24.8667,113.6333,clear sky,
4,Kudahuvadhoo,MV,81.55,2.6708,72.8944,overcast clouds,
9,Mar Del Plata,AR,69.82,-38.0023,-57.5575,clear sky,
10,Tiarei,PF,86.02,-17.5333,-149.3333,light rain,
11,Nong Chik,TH,83.08,6.8436,101.1780,overcast clouds,
15,Mogadishu,SO,77.43,2.0371,45.3438,overcast clouds,
17,Luganville,VU,80.06,-15.5333,167.1667,overcast clouds,
21,Salinopolis,BR,78.91,-0.6136,-47.3561,light rain,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():   

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Rikitea,PF,78.26,-23.1203,-134.9692,light rain,People ThankYou
3,Carnarvon,AU,77.07,-24.8667,113.6333,clear sky,Hospitality Carnarvon
4,Kudahuvadhoo,MV,81.55,2.6708,72.8944,overcast clouds,Niyama Private Islands Maldives
9,Mar Del Plata,AR,69.82,-38.0023,-57.5575,clear sky,Gran Hotel Mar del Plata
10,Tiarei,PF,86.02,-17.5333,-149.3333,light rain,Le Rocher de Tahiti


In [22]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# Configure gmaps API key
#gmaps.configure(api_key=g_key)

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))